In [52]:
import pandas as pd
from datetime import timedelta

In [53]:
# Read data from excel file

xlsx = r"C:\Users\mfaraday01\Documents\Code\Aging Analysis\credit_debit_records.xlsx"
debit = pd.read_excel (xlsx, 
                     sheet_name="Debits"
                     )
credit = pd.read_excel (xlsx, 
                     sheet_name="Credit"
                     )

In [54]:
# Unpivot "debit"

debit_unpiv = debit.melt(id_vars=["Zone", "Region", "District"], var_name="Date", value_name="Balance")

In [55]:
# Change datatypes

debit_unpiv[["Zone", "Region", "District"]] = debit_unpiv[["Zone", "Region", "District"]].astype(str)
debit_unpiv['Date'] = pd.to_datetime(debit_unpiv['Date']).dt.date

,Zone,Region,District,Date,Balance
0,1,1,Agona Amenfi,2021-10-01,50
1,1,1,Asankragwa A,2021-10-01,200
